# Robustness by regularization


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np
from Robust_nn.OrdOneL import OrdOneL, OrdTwoL
import matplotlib.pyplot as plt
from utils.utils import read_vision_dataset
from utils.convnet import ConvNet
from utils.convnet_silu import ConvNetSiLU
import os
import torch
import gc

**Read the DataLoader**

In [ ]:
trainloader, testloader = read_vision_dataset('../data', dataset='MNIST')

**Creating and comparing results**

In [ ]:
# options = {'only o1':(True,False, False), 'only o2':(False, True, False), 'both': (True,True, False), 'none':(False,False,False), 'modO2':(False,True,True) } 
options = {'only o1':(True,False, False), 'both': (True,True, True), 'none':(False,False,False) } 
rvec = [2,4,np.inf]
deltav = [0.2]
mdict = {}
basepath = os.curdir
mpath = os.path.join(basepath,'models')

for i,(k,(o1,o2, mod_o2)) in enumerate(options.items()):
  auxd = {}
  for r in rvec:
    rstr = str(r)
    print('\n*******\n ** Case '+k+', r=',r,'\n ******')
    torch.manual_seed(0)
    np.random.seed(0)
    auxd[rstr] = {}
    network = ConvNet()
    net_RegTrin = OrdTwoL(network, trainloader, testloader,  device='cuda', delta =0.2, r= r, o2=o2, o1=o1, mod_o2=mod_o2) #'cuda'
    net_RegTrin.set_optimizer(optim_alg='Adam', args={'lr':1e-4})
    net_RegTrin.train(epochs=5, delta=deltav)
    auxd[rstr]['train_loss'] = net_RegTrin.train_loss.copy()
    auxd[rstr]['train_acc'] = net_RegTrin.train_acc.copy()
    auxd[rstr]['train_reg'] = net_RegTrin.train_reg.copy()
    auxd[rstr]['test_acc_adv'] = net_RegTrin.test_acc_adv.copy()
    auxd[rstr]['test_acc_clean'] = net_RegTrin.test_acc_clean.copy()
    auxd[rstr]['train_times'] = net_RegTrin.train_times.copy()
    torch.save(network.state_dict(), os.path.join(mpath, k+'_r_'+str(r)+'.pth' ))
    del(network)
    gc.collect()
    torch.cuda.empty_cache()
  mdict[k] = auxd
  with open('tests_all.txt','w') as data: 
      data.write(str(mdict))